# Setup

## python version

In [4]:
from platform import python_version
print(python_version())

3.7.9


## Librerie

In [45]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mechanicalsoup as ms
import scrapy
import selenium
import lxml
import re
import requests
import pandas as pd
from pandas import compat
import json
from requests import get
import pprint
#import html5lib

# Tables 

## POKEDEX: Pokemon list with stats (id, nome, tipo, total, hp, attack, defense, sp. atk., sp. def, speed)

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [37]:
pokedex_URL = "https://pokemondb.net/pokedex/all"
pokedex_page = requests.get(pokedex_URL)

In [38]:
pokedex_soup = BeautifulSoup(pokedex_page.content, features="html.parser")

In [39]:
pokedex_table = pokedex_soup.find(id="pokedex")

In [40]:
pokedex_df = pd.read_html(str(pokedex_table))[0]

In [41]:
pokedex_name_split = pokedex_df['Name'].str.split(n = 1, expand = True)
pokedex_df['Name'] = pokedex_name_split[0]
pokedex_df.insert(2, "Variant", pokedex_name_split[1])
#pokedex_df['Type'] = pokedex_df['Type'].replace(' ', ', ', regex = True)

In [42]:
pokedex_type_split = pokedex_df['Type'].str.split(n = 1, expand = True)
pokedex_df.drop('Type', inplace = True, axis = 1)
pokedex_df.insert(2, "Type1", pokedex_type_split[0])
pokedex_df.insert(3, "Type2", pokedex_type_split[1])

In [43]:
pokedex_df

,#,Name,Type1,Type2,Variant,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,None,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,None,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,None,525,80,82,83,100,100,80
3,3,Venusaur,Grass,Poison,Mega Venusaur,625,80,100,123,122,120,80
4,4,Charmander,Fire,None,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion,Water,Ghost,Female,530,120,92,65,100,75,78
1071,903,Sneasler,Poison,Fighting,None,510,80,130,60,40,80,120
1072,904,Overqwil,Dark,Poison,None,510,85,115,95,65,65,85
1073,905,Enamorus,Fairy,Flying,Incarnate Forme,580,74,115,70,135,80,106


In [53]:
pokedex_json = json.dumps(list(pokedex_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4)

In [55]:
print(pokedex_json)

[
    {
        "#": 1,
        "Name": "Bulbasaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Total": 318,
        "HP": 45,
        "Attack": 49,
        "Defense": 49,
        "Sp. Atk": 65,
        "Sp. Def": 65,
        "Speed": 45
    },
    {
        "#": 2,
        "Name": "Ivysaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Total": 405,
        "HP": 60,
        "Attack": 62,
        "Defense": 63,
        "Sp. Atk": 80,
        "Sp. Def": 80,
        "Speed": 60
    },
    {
        "#": 3,
        "Name": "Venusaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Total": 525,
        "HP": 80,
        "Attack": 82,
        "Defense": 83,
        "Sp. Atk": 100,
        "Sp. Def": 100,
        "Speed": 80
    },
    {
        "#": 3,
        "Name": "Venusaur",
        "Type1": "Grass",
        "Type2": "Poison",
        "Variant": "Mega Venusaur",
        "Total": 625,
        "HP": 80,
        "Attack": 100,
        "De

Salvataggio

In [69]:
# JSON
with open('./Scraping_datasets/pokedex.json', 'w', encoding='utf8') as outfile:
    outfile.write(pokedex_json)
# CSV
pokedex_df.to_csv('./Scraping_datasets/pokedex.csv', na_rep="NA")

In [25]:
#pokedex_json = pokedex_df.to_json(orient='index', indent = 4)
#print(pokedex_json)

#  Items (Name, Category, Effect)

In [56]:
items_URL = "https://pokemondb.net/item/all"

In [58]:
items_page = requests.get(items_URL)

In [59]:
items_soup = BeautifulSoup(items_page.content, parser = 'html.parser')

In [60]:
items_df = pd.read_html(str(items_soup))[0]

In [61]:
items_df

,Name,Category,Effect
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...
2,Ability Urge,Battle items,"When used, it activates the Ability of an ally..."
3,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.
4,Absolite,Hold items,Enables Absol to Mega Evolve during battle.
...,...,...,...
955,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.
956,Yellow Shard,General items,A small yellow shard. It appears to be from so...
957,Zap Plate,Hold items,Increases power of Electric-type moves. Change...
958,Zinc,Medicine,Increases Special Defense EVs by 10.


In [74]:
items_json = json.dumps(list(items_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)

In [75]:
print(items_json)

[
    {
        "Name": "Ability Capsule",
        "Category": "Hold items",
        "Effect": "A capsule that allows a Pokémon with two Abilities to switch between these Abilities when it is used."
    },
    {
        "Name": "Ability Patch",
        "Category": "General items",
        "Effect": "Changes a Pokémon's ability to its Hidden Ability."
    },
    {
        "Name": "Ability Urge",
        "Category": "Battle items",
        "Effect": "When used, it activates the Ability of an ally Pokémon."
    },
    {
        "Name": "Abomasite",
        "Category": "Hold items",
        "Effect": "Enables Abomasnow to Mega Evolve during battle."
    },
    {
        "Name": "Absolite",
        "Category": "Hold items",
        "Effect": "Enables Absol to Mega Evolve during battle."
    },
    {
        "Name": "Absorb Bulb",
        "Category": "Hold items",
        "Effect": "A consumable bulb. If the holder is hit by a Water-type move, its Sp. Atk will rise."
    },
    {
        "Na

Salvataggio

In [77]:
# JSON
with open('./Scraping_datasets/items.json', 'w', encoding='utf8') as outfile:
    outfile.write(items_json)
# CSV
items_df.to_csv('./Scraping_datasets/items.csv', na_rep="NA")

## abilities (name, n_pokemon, description, Gen)

In [83]:
abilities_URL = "https://pokemondb.net/ability"
abilities_page = requests.get(abilities_URL)
abilities_soup = BeautifulSoup(abilities_page.content, parser = 'html.parser')
abilities_df = pd.read_html(str(abilities_soup))[0]
abilities_json = json.dumps(list(abilities_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/abilities.json', 'w', encoding='utf8') as outfile:
    outfile.write(abilities_json)
# CSV
abilities_df.to_csv('./Scraping_datasets/abilities.csv', na_rep="NA")

In [79]:
abilities_df

,Name,Pokémon,Description,Gen.
0,Adaptability,16,Powers up moves of the same type.,4
1,Aerilate,2,Turns Normal-type moves into Flying-type moves.,6
2,Aftermath,10,Damages the attacker landing the finishing hit.,4
3,Air Lock,1,Eliminates the effects of weather.,3
4,Analytic,12,Boosts move power when the Pokémon moves last.,5
...,...,...,...,...
261,White Smoke,4,Prevents other Pokémon from lowering its stats.,3
262,Wimp Out,1,Switches out when HP drops below half.,7
263,Wonder Guard,1,Only supereffective moves will hit.,3
264,Wonder Skin,5,Makes status-changing moves more likely to miss.,5


In [82]:
print(abilities_json)

[
    {
        "Name": "Adaptability",
        "Pokémon": 16,
        "Description": "Powers up moves of the same type.",
        "Gen.": 4
    },
    {
        "Name": "Aerilate",
        "Pokémon": 2,
        "Description": "Turns Normal-type moves into Flying-type moves.",
        "Gen.": 6
    },
    {
        "Name": "Aftermath",
        "Pokémon": 10,
        "Description": "Damages the attacker landing the finishing hit.",
        "Gen.": 4
    },
    {
        "Name": "Air Lock",
        "Pokémon": 1,
        "Description": "Eliminates the effects of weather.",
        "Gen.": 3
    },
    {
        "Name": "Analytic",
        "Pokémon": 12,
        "Description": "Boosts move power when the Pokémon moves last.",
        "Gen.": 5
    },
    {
        "Name": "Anger Point",
        "Pokémon": 9,
        "Description": "Maxes Attack after taking a critical hit.",
        "Gen.": 4
    },
    {
        "Name": "Anticipation",
        "Pokémon": 14,
        "Description": "Senses

## Pokémon EVs (Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed)

In [84]:
EV_URL = "https://pokemondb.net/ev/all"
EV_page = requests.get(EV_URL)
EV_soup = BeautifulSoup(EV_page.content, parser = 'html.parser')
EV_df = pd.read_html(str(EV_soup))[0]
EV_json = json.dumps(list(EV_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/EV.json', 'w', encoding='utf8') as outfile:
    outfile.write(EV_json)
# CSV
EV_df.to_csv('./Scraping_datasets/EV.csv', na_rep="NA")

In [85]:
EV_df

,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,NaN,NaN,NaN,1.0,NaN,NaN
1,2,Ivysaur,NaN,NaN,NaN,1.0,1.0,NaN
2,3,Venusaur,NaN,NaN,NaN,2.0,1.0,NaN
3,3,Venusaur Mega Venusaur,NaN,NaN,NaN,2.0,1.0,NaN
4,4,Charmander,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
1071,903,Sneasler,NaN,NaN,NaN,NaN,NaN,NaN
1072,904,Overqwil,NaN,NaN,NaN,NaN,NaN,NaN
1073,905,Enamorus Incarnate Forme,NaN,NaN,NaN,NaN,NaN,NaN
1074,905,Enamorus Therian Forme,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
print(EV_json)

[
    {
        "#": 1,
        "Name": "Bulbasaur",
        "Sp. Atk": 1.0
    },
    {
        "#": 2,
        "Name": "Ivysaur",
        "Sp. Atk": 1.0,
        "Sp. Def": 1.0
    },
    {
        "#": 3,
        "Name": "Venusaur",
        "Sp. Atk": 2.0,
        "Sp. Def": 1.0
    },
    {
        "#": 3,
        "Name": "Venusaur Mega Venusaur",
        "Sp. Atk": 2.0,
        "Sp. Def": 1.0
    },
    {
        "#": 4,
        "Name": "Charmander",
        "Speed": 1.0
    },
    {
        "#": 5,
        "Name": "Charmeleon",
        "Sp. Atk": 1.0,
        "Speed": 1.0
    },
    {
        "#": 6,
        "Name": "Charizard",
        "Sp. Atk": 3.0
    },
    {
        "#": 6,
        "Name": "Charizard Mega Charizard X",
        "Sp. Atk": 3.0
    },
    {
        "#": 6,
        "Name": "Charizard Mega Charizard Y",
        "Sp. Atk": 3.0
    },
    {
        "#": 7,
        "Name": "Squirtle",
        "Defense": 1.0
    },
    {
        "#": 8,
        "Name": "Wartortle",


# Moves

Name
Type
Cat.
Power
Acc.
PP
TM
Effect
Prob. (%))

In [87]:
move_URL = "https://pokemondb.net/move/all"
move_page = requests.get(move_URL)
move_soup = BeautifulSoup(move_page.content, parser = 'html.parser')
move_df = pd.read_html(str(move_soup))[0]
move_json = json.dumps(list(move_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/moves.json', 'w', encoding='utf8') as outfile:
    outfile.write(move_json)
# CSV
move_df.to_csv('./Scraping_datasets/moves.csv', na_rep="NA")

In [88]:
move_df

,Name,Type,Cat.,Power,Acc.,PP,TM,Effect,Prob. (%)
0,"10,000,000 Volt Thunderbolt",Electric,NaN,195,—,1,NaN,Pikachu-exclusive Z-Move. High critical hit ra...,—
1,Absorb,Grass,NaN,20,100,25,NaN,User recovers half the HP inflicted on opponent.,—
2,Accelerock,Rock,NaN,40,100,20,NaN,User attacks first.,—
3,Acid,Poison,NaN,40,100,30,NaN,May lower opponent's Special Defense.,10
4,Acid Armor,Poison,NaN,—,—,20,NaN,Sharply raises user's Defense.,—
...,...,...,...,...,...,...,...,...,...
860,Yawn,Normal,NaN,—,—,10,NaN,Puts opponent to sleep in the next turn.,—
861,Zap Cannon,Electric,NaN,120,50,5,NaN,Paralyzes opponent.,100
862,Zen Headbutt,Psychic,NaN,80,90,15,NaN,May cause flinching.,20
863,Zing Zap,Electric,NaN,80,100,10,NaN,May cause flinching.,30


In [91]:
print(move_json)

[
    {
        "Name": "10,000,000 Volt Thunderbolt",
        "Type": "Electric",
        "Power": "195",
        "Acc.": "—",
        "PP": "1",
        "Effect": "Pikachu-exclusive Z-Move. High critical hit ratio.",
        "Prob. (%)": "—"
    },
    {
        "Name": "Absorb",
        "Type": "Grass",
        "Power": "20",
        "Acc.": "100",
        "PP": "25",
        "Effect": "User recovers half the HP inflicted on opponent.",
        "Prob. (%)": "—"
    },
    {
        "Name": "Accelerock",
        "Type": "Rock",
        "Power": "40",
        "Acc.": "100",
        "PP": "20",
        "Effect": "User attacks first.",
        "Prob. (%)": "—"
    },
    {
        "Name": "Acid",
        "Type": "Poison",
        "Power": "40",
        "Acc.": "100",
        "PP": "30",
        "Effect": "May lower opponent's Special Defense.",
        "Prob. (%)": "10"
    },
    {
        "Name": "Acid Armor",
        "Type": "Poison",
        "Power": "—",
        "Acc.": "—",
      

# Natures and berries (nature_name, Increases, Decreases, Berrie_like, berrie_dislike)

In [100]:
natures_URL = "https://pokemondb.net/mechanics/natures"
natures_page = requests.get(natures_URL)
natures_soup = BeautifulSoup(natures_page.content, parser = 'html.parser')
natures_df = pd.read_html(str(natures_soup))[1]
natures_df

,Nature,Increases,Decreases
0,Adamant,Attack,Sp. Atk
1,Bashful,Sp. Atk,Sp. Atk
2,Bold,Defense,Attack
3,Brave,Attack,Speed
4,Calm,Sp. Def,Attack
5,Careful,Sp. Def,Sp. Atk
6,Docile,Defense,Defense
7,Gentle,Sp. Def,Defense
8,Hardy,Attack,Attack
9,Hasty,Speed,Defense


In base alla natura si hanno bacche che il pokemon apprezza disprezzza

In [101]:
def berries_taste(x):
    if x == 'Attack':
        val = 'Spicy'
    elif x == 'Defense':
        val = 'Sour'
    elif x == 'Speed':
        val = 'Sweet'
    elif x == 'Sp. Atk':
        val = 'Dry'
    elif x == 'Sp. Def': 
        val = 'Bitter'
    return val

In [102]:
natures_df['Likes_berrie'] = natures_df['Increases'].apply(berries_taste)
natures_df['Dislikes_berrie'] = natures_df['Decreases'].apply(berries_taste)

In [103]:
natures_df

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy
5,Careful,Sp. Def,Sp. Atk,Bitter,Dry
6,Docile,Defense,Defense,Sour,Sour
7,Gentle,Sp. Def,Defense,Bitter,Sour
8,Hardy,Attack,Attack,Spicy,Spicy
9,Hasty,Speed,Defense,Sweet,Sour


In [104]:
natures_json = json.dumps(list(natures_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/natures.json', 'w', encoding='utf8') as outfile:
    outfile.write(natures_json)
# CSV
natures_df.to_csv('./Scraping_datasets/natures.csv', na_rep="NA")